<a href="https://colab.research.google.com/github/Raoufmamedov/PET_PROJECTS/blob/main/Parsing_and_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q feedparser requests beautifulsoup4 pandas transformers spacy sacremoses
!python -m spacy download en_core_web_sm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import feedparser
import requests
from bs4 import BeautifulSoup
import pandas as pd
from transformers import pipeline
import spacy


In [28]:
import feedparser
import requests
from bs4 import BeautifulSoup
import pandas as pd
from transformers import pipeline
import spacy

class NewsCollector:
    def __init__(self):
        self.sources = [
            'https://habr.com/en/rss/hubs/machine_learning/articles/all/',
        ]
        self.nlp = spacy.load("en_core_web_sm")

    def extract_content(self, url):
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')
            content = ' '.join([p.get_text() for p in paragraphs])
            return content if content else "Не удалось извлечт контент."
        except requests.exceptions.RequestException as e:
            print(f"Ошибка выборки {url}: {e}")
            return "Не удалось извлечт контент."

    def extract_entities(self, text):
        if text and len(text) > 0:
            doc = self.nlp(text)
            entities = [(ent.text, ent.label_) for ent in doc.ents]
            return entities
        return []

    def collect_articles(self):
        articles = []
        for source in self.sources:
            feed = feedparser.parse(source)
            for entry in feed.entries:
                article = {
                    'title': entry.title,
                    'content': self.extract_content(entry.link),
                    'published': entry.published,
                    'source': source,
                    'url': entry.link
                }
                articles.append(article)
        return pd.DataFrame(articles)

    def preprocess_text(self, df):
        df['content_truncated'] = df['content'].str[:500]
        df['entities'] = df['content'].apply(self.extract_entities)

        return df
nc=NewsCollector()

In [35]:
df = nc.preprocess_text(nc.collect_articles())
df['entities'] = df['content'].apply(nc.extract_entities)

In [38]:
df['entities']

,entities
0,"[(AI, GPE), (AI, GPE), (2025, DATE), (Ontologi..."
1,"[(These days, DATE), (AI, GPE), (AI, GPE), (AI..."
2,"[(CAPTCHA, ORG), (QA, ORG), (CAPTCHA, ORG), (O..."
3,"[(Verilog Meetup, PRODUCT), (two, CARDINAL), (..."
4,"[(Google, ORG), (Time, ORG), (AI, GPE), (hours..."
5,"[(ERT, ORG), (Logistic, LOC), (ERT, ORG), (ERT..."
6,"[(the past two years, DATE), (AI, GPE), (multi..."
7,"[(Paul Karol, PERSON), (Russian, NORP), (two, ..."
8,"[(one, CARDINAL), (Reddit, GPE), (years ago, D..."
9,"[(ChatGPT-4, ORG), (OpenAI, GPE), (20, MONEY),..."
